In [6]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00


## 🖥️ Importing Libraries¶


In [42]:
# Import pandas, a library for data analysis and manipulation
import pandas as pd

import json

import sys

import gc

from sklearn.model_selection import StratifiedKFold

import numpy as np

from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset

from tqdm.auto import tqdm

from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier

from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import VotingClassifier

In [43]:
from google.colab import files
uploaded = files.upload()

In [26]:
from google.colab import files
uploaded = files.upload()

Saving train_essays.csv to train_essays.csv
Saving train_prompts.csv to train_prompts.csv
Saving train_v2_drcat_02.csv to train_v2_drcat_02.csv


In [30]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [46]:

test = pd.read_csv('/content/test_essays.csv')

sub = pd.read_csv('/content/sample_submission.csv')

train = pd.read_csv('/content/train_v2_drcat_02.csv', sep=',')


In [47]:
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)


In [50]:
# This means that the text will not be converted to lowercase before tokenization.
LOWERCASE = False

# The following line of code sets the VOCAB_SIZE to 14000000.
# This means that the maximum number of words in the vocabulary will be 14 million.
VOCAB_SIZE = 140000


In [51]:
# Create a tokenizer object using the Byte Pair Encoding (BPE) algorithm
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])

raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]

trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

dataset = Dataset.from_pandas(test[['text']])

def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

tokenized_texts_test = []

for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/44868 [00:00<?, ?it/s]

In [52]:
tokenized_texts_test[1]


['ĠBbb', 'Ġccc', 'Ġddd', '.']

## 📑 TF-IDF Vectorization

In [53]:
def dummy(text):
    return text

# Create a TfidfVectorizer object that will extract n-grams of words (3 to 5) from the text, without lowercasing or tokenizing it
vectorizer = TfidfVectorizer(ngram_range=(4, 4), lowercase=False, sublinear_tf=True, analyzer = 'word',
    tokenizer = dummy,
    preprocessor = dummy,
    token_pattern = None, strip_accents='unicode')

vectorizer.fit(tokenized_texts_test)

vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(4, 4), lowercase=False, sublinear_tf=True, vocabulary=vocab,
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode'
                            )

tf_train = vectorizer.fit_transform(tokenized_texts_train)

tf_test = vectorizer.transform(tokenized_texts_test)

del vectorizer
gc.collect()


{'ĠAaa Ġbbb Ġccc .': 0, 'ĠBbb Ġccc Ġddd .': 1, 'ĠCCC Ġddd Ġeee .': 2}


29

In [54]:
y_train = train['label'].values

## Model Training and Prediction¶


In [56]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 12.6 MB/s eta 0:00:00


In [57]:
# Define a function that returns an ensemble model of four classifiers
def get_model():

    from catboost import CatBoostClassifier

    # Create a Multinomial Naive Bayes classifier with a smoothing parameter of 0.0235
    clf = MultinomialNB(alpha=0.0235)

    sgd_model = SGDClassifier(max_iter=9000, tol=3e-4, loss="modified_huber", random_state=6743)

    # Define a dictionary of parameters for a LightGBM classifier
    p6={'n_iter': 3000,'verbose': -1,'objective': 'cross_entropy','metric': 'auc',
        'learning_rate': 0.0031909898961407, 'colsample_bytree': 0.78,
        'colsample_bynode': 0.8,
       }

    p6["random_state"] = 6743

    lgb=LGBMClassifier(**p6)

    # Create a CatBoost classifier with 3000 iterations, a learning rate of 0.003599066836106983, a subsample of 0.4, a cross entropy loss function, and a random seed of 6543
    cat=CatBoostClassifier(iterations=3000,
                           verbose=0,
                           random_seed=6543,
                           learning_rate=0.003599066836106983,
                           subsample = 0.4,
                           allow_const_label=True,loss_function = 'CrossEntropy')

    weights = [0.1,0.31,0.31,0.6]

    ensemble = VotingClassifier(estimators=[('mnb',clf),
                                            ('sgd', sgd_model),
                                            ('lgb',lgb),
                                            ('cat', cat)
                                           ],
                                weights=weights, voting='soft', n_jobs=-1)

    return ensemble

model = get_model()

print(model)

# Check the length of the test text values
if len(test.text.values) <= 5:
    sub.to_csv('submission.csv', index=False)
else:
    model.fit(tf_train, y_train)

    gc.collect()

    final_preds = model.predict_proba(tf_test)[:,1]

    sub['generated'] = final_preds
    sub.to_csv('submission.csv', index=False)
    sub


VotingClassifier(estimators=[('mnb', MultinomialNB(alpha=0.0235)),
                             ('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=9000, random_state=6743,
                                            tol=0.0003)),
                             ('lgb',
                              LGBMClassifier(colsample_bynode=0.8,
                                             colsample_bytree=0.78,
                                             learning_rate=0.0031909898961407,
                                             metric='auc', n_iter=3000,
                                             objective='cross_entropy',
                                             random_state=6743, verbose=-1)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7ce5047c24d0>)],
                 n_jobs=-1, voting='soft', weights=[0.1, 0.31, 0.31, 0.6])
